In [1]:
from pyspark.sql import functions as fun
from pyspark.sql.functions import col
from pyspark import Row
from pyspark.sql.types import StringType
from pyspark.sql import SQLContext
from pyspark.sql.dataframe import DataFrame

In [2]:
sqlContext = SQLContext(sc)

In [3]:
new_df = sqlContext\
                .createDataFrame(sc.parallelize(['Games', 'Food', 'Music']), StringType())\
                .selectExpr('value as category')
new_df.show()

+--------+
|category|
+--------+
|   Games|
|    Food|
|   Music|
+--------+



In [4]:
ks_df = sqlContext.read.csv('ks-projects-201612.csv', header=True)

ks_df = ks_df.select(ks_df.columns[:-4])
coluns = ks_df.columns[:]
for c in coluns:
    ks_df = ks_df.withColumnRenamed(c, c.strip())
ks_df.limit(5).toPandas().head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000,2015-08-11 12:12:28,0,failed,0,GB,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000,2013-01-12 00:20:50,220,failed,3,US,220
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000,2012-03-17 03:24:11,1,failed,1,US,1
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500,2015-07-04 08:35:03,1283,canceled,14,US,1283
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000,2016-02-26 13:38:27,52375,successful,224,US,52375


In [5]:
df = new_df.join(ks_df.alias('ks'), new_df['category']==ks_df['category']).select(['ks.'+c for c in ks_df.columns])
df.limit(5).toPandas().head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 18:30:44,1000,2014-12-01 18:30:44,1205,successful,16,US,1205
1,1000071625,Boco Tea,Food,Food,USD,2012-06-02 17:24:32,5000,2012-05-03 17:24:32,1781,failed,40,US,1781
2,1000282287,Babe Ruth's Family Kitchen - Gourmet Hot Dogs ...,Food,Food,USD,2015-10-13 00:59:36,25000,2015-09-08 00:59:36,0,canceled,0,US,0
3,1000476123,Mama's Meatballs,Food,Food,USD,2014-08-07 18:52:26,100,2014-07-08 18:52:26,0,failed,0,US,0
4,1000834930,Strawberry Fields Caf� and Patisserie,Food,Food,CAD,2017-01-23 17:14:00,50000,2016-12-01 21:16:27,0,live,0,CA,0


# Which is the older project fully supported?

In [6]:
#df.where('state=="live"').sort("launched").limit(5).toPandas().head()
df.where('state=="successful"').sort("launched").limit(5).toPandas().head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,530647695,Late Night Tunes,Music,Music,USD,2009-07-01 23:13:00,500,2009-05-13 04:30:06,510,successful,38,US,510
1,755288436,Help me get equipment and I'll make a song for...,Music,Music,USD,2009-07-15 05:59:00,360,2009-06-01 03:37:06,360,successful,14,US,360
2,665820533,Produce JigGsaw's new EP,Music,Music,USD,2009-08-12 23:00:00,800,2009-06-01 09:58:42,1057.22,successful,16,US,1057.22
3,1525594295,"""The RAA 7"""" Project""",Music,Music,USD,2009-08-02 04:26:00,5000,2009-06-05 00:08:37,6700,successful,104,US,6700
4,1756613231,"Volcano, I'm Still Excited!!: Help press a li...",Music,Music,USD,2009-09-09 00:00:00,3500,2009-06-11 22:59:42,3622,successful,84,US,3622


# Which is the main category with the most pledged ammount?

In [7]:
df.groupBy('main_category').agg(fun.sum('usd pledged')).toPandas().head()

,main_category,sum(usd pledged)
0,Food,6.077312e+07
1,Games,1.761419e+07
2,Music,5.164571e+07


# What is the average pledged ammount in USD for each category?

In [8]:
df.groupBy("category")\
        .agg({'category': 'count', 'usd pledged': 'sum'})\
        .withColumn('Mean', col('sum(usd pledged)')/col('count(category)'))\
        .toPandas().head()

,category,sum(usd pledged),count(category),Mean
0,Food,6.077312e+07,10516,5779.109584
1,Games,1.761419e+07,2676,6582.283384
2,Music,5.164571e+07,13841,3731.356551


# Save the US projects from the categories listed on our created dataframe in a csv file

In [9]:
us = df.where('country=="US"')
us.limit(5).toPandas().head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged
0,1000023410,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 18:30:44,1000,2014-12-01 18:30:44,1205,successful,16,US,1205
1,1000071625,Boco Tea,Food,Food,USD,2012-06-02 17:24:32,5000,2012-05-03 17:24:32,1781,failed,40,US,1781
2,1000282287,Babe Ruth's Family Kitchen - Gourmet Hot Dogs ...,Food,Food,USD,2015-10-13 00:59:36,25000,2015-09-08 00:59:36,0,canceled,0,US,0
3,1000476123,Mama's Meatballs,Food,Food,USD,2014-08-07 18:52:26,100,2014-07-08 18:52:26,0,failed,0,US,0
4,100101389,"Uptown Juice Company: Local, cold-pressed good...",Food,Food,USD,2014-04-01 08:59:00,25000,2014-02-25 00:15:17,30066,successful,161,US,30066


In [10]:
us.repartition(1).write.format('csv').save('us_csv')

# Tell us how would you save these entries on an Impala database

In [11]:
##passing the jdbc on context or with hive enable, but I'm not sure
# df.write.mode("overwrite").saveAsTable("database_name.table_name") 